In [4]:
import tensorflow as tf
import timeit
from datetime import datetime

In [2]:
# What is a graph?
# eager execution vs graph execution

# eager execution : runs on python, returning results to python
# graph execution : enables portability outside pythons such as mobile devices, better performance

# graphs : datastructure that contains multiple tf.Operation and tf.Tensor objects
# tensorflow uses graphs to export models from python

# graphs can be optimized by compilers such as
# "constant folidng" , "allocating sub-part computations to threads or devices"

In [6]:
# How to use graphs


# define a python fuction
def a_regular_fucntion(x,y,b):
    x = tf.matmul(x,y)
    x = x + b
    return x


a_graph_function = tf.function(a_regular_fucntion)

x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = a_regular_fucntion(x1, y1, b1).numpy()
graph_value = a_graph_function(x1,y1,b1).numpy()

assert(orig_value == graph_value)

In [8]:
def inner_function(x,y,b):
    x = tf.matmul(x,y)
    x = x + b
    return x

@tf.function
def outer_function(x):
    y = tf.constant([[2.0],[3.0]])
    b = tf.constant(4.0)

    return inner_function(x,y,b)

outer_function(tf.constant([[1.0,2.0]])).numpy()

array([[12.]], dtype=float32)

In [12]:
def simple_relu(x):
    if tf.greater(x,0):
        return x
    else:
        return 0

tf_simple_relu = tf.function(simple_relu)

print(tf_simple_relu(tf.constant(1)).numpy())
print(simple_relu(tf.constant(1)).numpy())

# inspect generated graph directly

print(tf.autograph.to_code(simple_relu))
print(tf_simple_relu.get_concrete_function(tf.constant(1)).graph.as_graph_def())

1
1
def tf__simple_relu(x):
    with ag__.FunctionScope('simple_relu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal retval_, do_return
            (do_return, retval_) = vars_

        def if_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = ag__.ld(x)
            except:
                do_return = False
                raise

        def else_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = 0
            except:
                do_return = False
                raise
        ag__.if_stmt(ag__.converted_call(ag__.ld(tf).greater, (ag__.ld(x), 0), None, fscope), i

In [14]:
# Ploymorphism
# each time when invoking Function with different dtypes
# will generate new tf.Graph

@tf.function
def my_relu(x):
    return tf.maximum(0., x)

# each time will create new graph (because of different dtypes)
print(my_relu(tf.constant(5.5)))
print(my_relu([-1,1]))
print(my_relu(tf.constant([-3.0, 3.0])))

# Signature matches with graph used before, not creating new graphs
print(my_relu(tf.constant(-2.5)))
print(my_relu(tf.constant([1.0,-1.0])))

tf.Tensor(5.5, shape=(), dtype=float32)
tf.Tensor([0. 1.], shape=(2,), dtype=float32)
tf.Tensor([0. 3.], shape=(2,), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor([1. 0.], shape=(2,), dtype=float32)


In [18]:
@tf.function
def get_MSE(y_true, y_pred):
    sq_diff = tf.pow(y_true - y_pred, 2)
    return tf.reduce_mean(sq_diff)

y_true = tf.random.uniform([5], maxval = 10, dtype = tf.int32)
y_pred = tf.random.uniform([5], maxval = 10, dtype = tf.int32)

print(y_true)
print(y_pred)

get_MSE(y_true, y_pred)

# tf.config.run_functions_eagerly(True) makes functions execute without creating graphs

tf.config.run_functions_eagerly(True)
get_MSE(y_true, y_pred)


tf.Tensor([3 3 9 9 1], shape=(5,), dtype=int32)
tf.Tensor([8 1 9 1 1], shape=(5,), dtype=int32)


<tf.Tensor: shape=(), dtype=int32, numpy=18>

In [24]:
# Function can behave different with certain operations
tf.config.run_functions_eagerly(False)

@tf.function
def get_MSE(y_true, y_pred):
    print("Calculating MSE")
    sq_diff = tf.pow(y_true - y_pred, 2)
    return tf.reduce_mean(sq_diff)

error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

print("print() only executed once becuse not traced\n")
tf.config.run_functions_eagerly(True)

error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

print("print() executed three times with eager execution")
tf.config.run_functions_eagerly(False)


Calculating MSE
print() only executed once becuse not traced

Calculating MSE
Calculating MSE
Calculating MSE
print() executed three times with eager execution


In [28]:
# seeing speed-up

x = tf.random.uniform(shape=[10,10], minval= -1, maxval = 2, dtype= tf.dtypes.int32)

def power(x,y):
    result = tf.eye(10, dtype = tf.dtypes.int32)
    for _ in range(y):
        result = tf.matmul(x,result)
    return result

print("Eager execution : ", timeit.timeit(lambda : power(x,100), number=1000))

power_as_graph = tf.function(power)
print("Graph execution: ", timeit.timeit(lambda : power_as_graph(x,100), number=1000))

Eager execution :  2.8066922000007253
Graph execution:  0.5774352999997063


In [30]:
@tf.function
def a_function_with_python_side_effect(x):
    print("Tracing")
    return x * x + tf.constant(2)

print(a_function_with_python_side_effect(tf.constant(2)))
print(a_function_with_python_side_effect(tf.constant(3)))

print(a_function_with_python_side_effect(2))
print(a_function_with_python_side_effect(3))


Tracing
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(11, shape=(), dtype=int32)
Tracing
tf.Tensor(6, shape=(), dtype=int32)
Tracing
tf.Tensor(11, shape=(), dtype=int32)
